In [ ]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import PrintfTickFormatter
from bokeh.io import output_notebook, output_file
from bokeh.plotting import figure, show, save, ColumnDataSource
from bokeh.models import PrintfTickFormatter, NumeralTickFormatter, HoverTool, Range1d
from pymongo import MongoClient
from functools import reduce
import statistics
import random
import numpy as np
import scipy as sp
import bokeh
import math
import re
from bson import json_util
import json
shuffled_colors = ['black','blue','green','purple','red','orange','silver','yellow','brown','magenta','cyan','olive']*10

factor = 0.9
plot_width = round(1000*factor)
plot_height = round(550*factor)

def get_distribution(things):
    distribution = {}
    for thing in things:
        if not thing in distribution:
            distribution[thing] = 0
        distribution[thing] += 1
    return distribution

def get_cumulative_distribution(dist):
    cumulative_distribution = {}
    value_from_previous = 0
    for thing in sorted(dist.keys()):
        cumulative_distribution[thing] = value_from_previous + dist[thing]
        value_from_previous = cumulative_distribution[thing]
mlab_rtts = db.baseline.find()
# print(mlab_rtts)
# data = []
# os_names = []
os_and_values = {}
for doc in mlab_rtts:
    if ('platform' in doc and 
    'rtt' in doc and 'rttx' in doc['rtt'] and 'result' in doc['rtt']['rttx'] and 'rtt' in doc['rtt']['rttx']['result'] and
    not 'dev' in doc['fathomversion']):
        if doc['platform'] in os_and_values:
            os_and_values[doc['platform']].extend(doc['rtt']['rttx']['result']['rtt'])
        else:
            os_and_values[doc['platform']] = doc['rtt']['rttx']['result']['rtt']    return cumulative_distribution

In [ ]:
mlab_rtts = db.baseline.find()
# print(mlab_rtts)
# data = []
# os_names = []
os_and_values = {}
for doc in mlab_rtts:
    if ('platform' in doc and 
    'rtt' in doc and 'rttx' in doc['rtt'] and 'result' in doc['rtt']['rttx'] and 'rtt' in doc['rtt']['rttx']['result'] and
    not 'dev' in doc['fathomversion']):
        if doc['platform'] in os_and_values:
            os_and_values[doc['platform']].extend(doc['rtt']['rttx']['result']['rtt'])
        else:
            os_and_values[doc['platform']] = doc['rtt']['rttx']['result']['rtt']

p_interface = figure(tools="pan,box_zoom,reset,save,wheel_zoom", plot_width=plot_width, plot_height=plot_height)
p_interface.add_tools(HoverTool(tooltips=[("(x,y)", "($x, $y)")]))
p_interface.xaxis.axis_label="x"
p_interface.yaxis.axis_label="y"
p_interface.xaxis.formatter = PrintfTickFormatter(format="%.2f ms")
p_interface.yaxis.formatter = NumeralTickFormatter(format="0.[00000]")
    
# p.line('x','y', source=ColumnDataSource(data=dict(x=dist['x'], y=dist['y'])))

things = os_and_values
for index, key in enumerate(things.keys()):
    just_dist = get_distribution([item for item in things[key] if item >= 0])

    # dist = sorted(just_dist.items())
    # dist = {'x': [item[0] for item in dist], 'y': [item[1] for item in dist]}

    cum_dist = sorted(get_cumulative_distribution(just_dist).items())
    cum_dist = {'x': [item[0] for item in cum_dist], 'y': [item[1] for item in cum_dist]}
    m = max(cum_dist['y'])
#     print('minimum x for', key, min(cum_dist['x']))
#     print('minimum y for', key, min(cum_dist['y']))
    cum_dist['y'] = [item/m for item in cum_dist['y']]

    p.line('x','y', legend=key, color=shuffled_colors[index], source=ColumnDataSource(data=dict(x=cum_dist['x'], y=cum_dist['y'])))

# output_notebook()
# show(p)

In [ ]:
p_hops = figure(tools="pan,box_zoom,reset,save,wheel_zoom", plot_width=plot_width, plot_height=plot_height)
p_hops.add_tools(HoverTool(tooltips=[("(x,y)", "($x, $y)")]))
p_hops.xaxis.axis_label="x"
p_hops.yaxis.axis_label="y"
p_hops.xaxis.formatter = PrintfTickFormatter(format="%d hops")
p_hops.yaxis.formatter = NumeralTickFormatter(format="0.[00000]")

hop_numbers = []
for item in db.baseline.find():
#     if 'traceroute' in item and not "missed" in as_json and not 'error' in as_json:
    if 'traceroute' in item and 'result' in item['traceroute'] and 'hops' in item['traceroute']['result'] and 0<len(item['traceroute']['result']['hops'])<30:
        as_json = json.dumps(item['traceroute'],default=json_util.default)
        if not 'error' in as_json:
            hop_numbers.append(len(item['traceroute']['result']['hops']))

just_dist = get_distribution(hop_numbers)

cum_dist = sorted(get_cumulative_distribution(just_dist).items())
cum_dist = {'x': [item[0] for item in cum_dist], 'y': [item[1] for item in cum_dist]}
m = max(cum_dist['y'])

cum_dist['y'] = [item/m for item in cum_dist['y']]

#     p[i].scatter('x', 'y', fill_alpha=alpha_color, color=shuffled_colors[things_to_extract.index(thing)], marker=markers_by_type[measurement_type], line_color=None, source=ColumnDataSource(data=dict(x=x[thing+','+measurement_type][i],y=y[thing+','+measurement_type][i],desc=[thing]*len(x[thing+','+measurement_type][i]))))
p_hops.line('x','y', color=shuffled_colors[index], source=ColumnDataSource(data=dict(x=cum_dist['x'], y=cum_dist['y'])))

output_notebook()
show(p_hops)

In [ ]:
# counter = 0
# already_found_environments = set()
# for item in db.baseline.find():
#     if counter >= 10:
#         break
#     if 'traceroute' in item and 'result' in item['traceroute'] and 'hops' in item['traceroute']['result'] and 0<len(item['traceroute']['result']['hops'])<30:
#         as_json = json.dumps(item['traceroute'],default=json_util.default)
#         if not 'error' in as_json and item['networkenv']['ssid']:# and item['networkenv']['default_iface_name'] not in already_found_environments:
# #             already_found_environments.add(item['networkenv']['default_iface_name'])
#             counter+=1
#             print('env_id', item['networkenv']['default_iface_name'])
#             print('__ITEM__', json.dumps(item['networkenv'],default=json_util.default))

In [ ]:
# counter = 0
# already_found_environments = set()

rtt_by_interface = {}
for item in db.baseline.find():
#     if counter >= 10:
#         break
    if 'traceroute' in item and 'result' in item['traceroute'] and 'hops' in item['traceroute']['result'] and 0<len(item['traceroute']['result']['hops'])<30:
        as_json = json.dumps(item['traceroute'],default=json_util.default)
        if not 'error' in as_json:
            if item['networkenv']['ssid'] or item['networkenv']['bssid']:
                key = 'wifi'
            else:
                key = 'wired'
            if key in rtt_by_interface:
                rtt_by_interface[key].extend(item['rtt']['rttx']['result']['rtt'])
            else:
                rtt_by_interface[key] = item['rtt']['rttx']['result']['rtt']
#             already_found_environments.add(item['networkenv']['default_iface_name'])
#             counter+=1
#             print('env_id', item['networkenv']['default_iface_name'])
#             print('__ITEM__', json.dumps(item,default=json_util.default))

p_interface = figure(tools="pan,box_zoom,reset,save,wheel_zoom", plot_width=plot_width, plot_height=plot_height)
p_interface.add_tools(HoverTool(tooltips=[("(x,y)", "($x, $y)")]))
p_interface.xaxis.axis_label="x"
p_interface.yaxis.axis_label="y"
p_interface.xaxis.formatter = PrintfTickFormatter(format="%.2f ms")
p_interface.yaxis.formatter = NumeralTickFormatter(format="0.[00000]")

things = rtt_by_interface
for index, key in enumerate(things.keys()):
    just_dist = get_distribution([item for item in things[key] if item >= 0])

    # dist = sorted(just_dist.items())
    # dist = {'x': [item[0] for item in dist], 'y': [item[1] for item in dist]}

    cum_dist = sorted(get_cumulative_distribution(just_dist).items())
    cum_dist = {'x': [item[0] for item in cum_dist], 'y': [item[1] for item in cum_dist]}
    m = max(cum_dist['y'])
#     print('minimum x for', key, min(cum_dist['x']))
#     print('minimum y for', key, min(cum_dist['y']))
    cum_dist['y'] = [item/m for item in cum_dist['y']]

    p_interface.line('x','y', legend=key, color=shuffled_colors[index], source=ColumnDataSource(data=dict(x=cum_dist['x'], y=cum_dist['y'])))

output_notebook()
show(p_interface)